<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyze search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


#### Install spark

In [1]:
!pip install pyspark
!pip install findspark

#### Start session

In [2]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Saving and Loading a SparkML Model").getOrCreate()

23/05/17 01:56:15 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/17 01:56:17 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


#### Download the search term dataset from the below url:
https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

#### Load the csv into a spark dataframe

In [3]:
import pandas as pd
searchterms = pd.read_csv("./searchterms.csv")
sdf = spark.createDataFrame(searchterms)

#### Print the number of rows and columns.  Take a screenshot of the code and name it as `shape.jpg`)

In [4]:
sdf.count(), len(sdf.columns)

(10000, 4)

#### Print the top 5 rows.  Take a screenshot of the code and name it as `top5rows.jpg`)

In [5]:
sdf.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



#### Find out the datatype of the column `searchterm`. Take a screenshot of the code and name it as `datatype.jpg`)

In [6]:
sdf.printSchema()

root
 |-- day: long (nullable = true)
 |-- month: long (nullable = true)
 |-- year: long (nullable = true)
 |-- searchterm: string (nullable = true)



#### How many times was the term `gaming laptop` searched?  Take a screenshot of the code and name it as `gaminglaptop.jpg`)

In [7]:
from pyspark.sql.functions import count

results = sdf.filter(sdf.searchterm == "gaming laptop").agg(count("*").alias("count"))
results.show()

[Stage 3:==============>                                            (2 + 6) / 8]

+-----+
|count|
+-----+
|  499|
+-----+



#### Print the top 5 most frequently used search terms. Take a screenshot of the code and name it as `top5terms.jpg`)

In [8]:
from pyspark.sql.functions import desc

results = sdf.groupBy("searchterm").agg(count("*").alias("count")).sort(desc("count"))
results.show(5)

[Stage 6:===================================================>  (189 + 10) / 200]

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



#### The pretrained sales forecasting model is available at this url:
https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

#### Load the sales forecast model. Take a screenshot of the code and name it as `loadmodel.jpg`)

In [9]:
from pyspark.ml.regression import LinearRegressionModel

model = LinearRegressionModel.load('sales_prediction.model')
model

LinearRegression_6d5736f3dbe7

#### Using the sales forecast model, predict the sales for the year of 2023. Take a screenshot of the code and name it as `forecast.jpg`

In [10]:
df = spark.read.parquet('sales_prediction.model/data/part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet')
df.printSchema()

root
 |-- intercept: double (nullable = true)
 |-- coefficients: vector (nullable = true)
 |-- scale: double (nullable = true)



In [11]:
# This function converts a scalar number into a dataframe that can be used by the model to predict.

from pyspark.ml.feature import VectorAssembler

def predict(year):
    assembler = VectorAssembler(inputCols=["year"],outputCol="features")
    data = [[0,0,year,0]]
    columns = ["day", "month", "year", "search"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features','year')
    predictions = model.transform(__)
    predictions.select('prediction').show()
    
predict(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



23/05/17 01:56:49 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/05/17 01:56:49 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
